In [10]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

## Task1

빈 칸을 채워주세요!

단계별 output이 github 파일에는 남아있으니 그 output과 동일한 형태인지 확인하면서 진행해주시면 됩니다~

In [ ]:
#1. 생성할 문장 데이터

sentence = ("Brick walls are there for a reason and you must not think "
            "that the brick walls aren't there to keep us out, but rather "
            "in this way that the brick walls are there to show us how badly we want things.")

In [ ]:
#2. 문자 집합 만들기
world_set = list(set(sentence)) # 중복 문자 제거

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
# 코드 시행할 때마다 인덱스 붙는게 계속 달라짐.
# world_set을 sorted() 함수를 써서 정렬하면 인덱스가 정해진 순서로 고정됨

#vocab = {##}
vocab = {char: idx for idx, char in enumerate(world_set)} # 딕셔너리 자료형에 key = 단어, value: 인덱스로 저장

In [ ]:
print(vocab)

{'a': 0, 'm': 1, 'r': 2, 'e': 3, 'k': 4, 'y': 5, 'o': 6, 'c': 7, 'b': 8, 's': 9, ',': 10, 'B': 11, "'": 12, 'g': 13, 'w': 14, 'u': 15, 'p': 16, 'f': 17, 'd': 18, 'h': 19, 'i': 20, ' ': 21, 'l': 22, 'n': 23, 't': 24, '.': 25}


In [ ]:
#3. 문자 집합 크기 확인

vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 26


In [ ]:
#4. 하이퍼 파라미터 설정(자유롭게 수정해보세요!) #일단 그대로 진행

hidden_size = vocab_size # 같아야 하는 것 확인!
sequence_length = 10  # 너무 길거나 너무 짧게 잡으면 안됩니다!
learning_rate = 0.01

In [ ]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length + 1 ]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 Brick wall -> rick walls
1 rick walls -> ick walls 
2 ick walls  -> ck walls a
3 ck walls a -> k walls ar
4 k walls ar ->  walls are
5  walls are -> walls are 
6 walls are  -> alls are t
7 alls are t -> lls are th
8 lls are th -> ls are the
9 ls are the -> s are ther
10 s are ther ->  are there
11  are there -> are there 
12 are there  -> re there f
13 re there f -> e there fo
14 e there fo ->  there for
15  there for -> there for 
16 there for  -> here for a
17 here for a -> ere for a 
18 ere for a  -> re for a r
19 re for a r -> e for a re
20 e for a re ->  for a rea
21  for a rea -> for a reas
22 for a reas -> or a reaso
23 or a reaso -> r a reason
24 r a reason ->  a reason 
25  a reason  -> a reason a
26 a reason a ->  reason an
27  reason an -> reason and
28 reason and -> eason and 
29 eason and  -> ason and y
30 ason and y -> son and yo
31 son and yo -> on and you
32 on and you -> n and you 
33 n and you  ->  and you m
34  and you m -> and you mu
35 and you mu -> nd you mus
36

In [ ]:
# 출력해서 한 칸씩 쉬프트된 것 확인하기!

# 코드 시행할 때마다 인덱스 붙는게 계속 달라짐.
# 이전 코드에서 world_set을 sorted() 함수를 써서 정렬하면 인덱스가 정해진 순서로 고정됨

print(x_data[0])
print(y_data[0])

[11, 2, 20, 7, 4, 21, 14, 0, 22, 22]
[2, 20, 7, 4, 21, 14, 0, 22, 22, 9]


In [ ]:
##6. 입력 시퀀스에 대해 원핫인코딩 수행

## 문제(4) : x_data를 원핫인코딩 > numpy의 eye를 쓸 수 있지 않을까?

x_data_np = np.array(x_data)
x_one_hot = np.eye(len(vocab))[x_data_np] #len(vocab) 크기의 identity matrix 형성. 인덱스별로 대응 -> one hot 인코딩!

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)## 신경망의 가중치와 편향이 일반적으로 부동소수점 형태로 저장되어 있음.
Y = torch.LongTensor(y_data) # 레이블은 주로 클래스 또는 카테고리를 나타냄, 어느정도 관례!

In [ ]:
##8. 크기 확인
print('훈련 데이터의 크기 : {}'.format(X.shape))
print('레이블의 크기 : {}'.format(Y.shape))

훈련 데이터의 크기 : torch.Size([188, 10, 26])
레이블의 크기 : torch.Size([188, 10])


In [ ]:
##9.원핫인코딩 결과 샘플 확인하기
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0.,

In [ ]:
##10. 레이블 데이터 샘플 확인하기
print(Y[0])

tensor([ 2, 20,  7,  4, 21, 14,  0, 22, 22,  9])


In [ ]:
##11. RNN 모델 구현

##문제(6) : 기본 pytorch 인자 넣기 연습 + forward 채우기
### 조건 : rnn layer 2개 쌓기 + 마지막은 fc layer
### batch_first 설정 필요할까? (유튜브 강의 참고)

class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim,
                            hidden_dim,
                            layers,
                            batch_first = True,
                            )
    self.fc = torch.nn.Linear(hidden_dim, input_dim) # ??? fully conneceted layer 차원 어떻게 구성하는지, 아직 원리 이해 x

  def forward(self, x):
    # First RNN layer
    x, _ = self.rnn(x)

    # Second RNN layer
    x, _ = self.rnn(x)

    # Fully connected layer
    x = self.fc(x)

    return x

In [ ]:
net = Net(vocab_size, hidden_size, 2)

In [ ]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
##14. 출력 크기 점검
outputs = net(X)
print(outputs.shape)

torch.Size([188, 10, 26])


In [ ]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    #loss = criterion(outputs.view(##, ##), Y.view(##))
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1)) # criterion 손실함수 - 크로스엔트로피
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

wwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwwww
       w w       w                                                    w   w w     w                                                            w   w w       w                w w                w   
                                                                                                                                                                                                     
                                                                                                                                                                                                     
                                                                                                                                                                                                     
          

In [ ]:
predict_str

"aeck walls are there tor a reason and you must not think that the brick walls are 't there to seep us out, but rather in this way that the brick walls are there to show us how badly we want thingsl"

결과가 어떤가요?? 마지막 에폭의 문장이 그럴싸한가요?

## Task2

위 sentence는 제가 임의로 생성한 문장들입니다.

마음에 드시는 문구 가져오셔서 문장이 어떻게 생성되는지 확인해보세요!

영어가 아닌 한국어로 시도해보는 것도 좋겠죠?

수정이 많이 필요(토큰화 등) 할 수 있으나 한번 시도해보시는 것 권장드립니다 :)

위 베이스라인은 어디든 수정하셔도 좋고 조금 더 자연스러운 문장이 나올 수 있게 다양한 시도를 해보세요!

조건 : 문장 3개 이상, 연결성이 있는 문장을 " " 으로 구분하여 ( )에 넣기

In [1]:
#스터디 참고하여 한국어로 시도!

sentence = ("사람은 누구나 몰입할 수 있는 능력을 가지고 있다."
            "위기 상황에서 몰입하기도 하고, 몰입이 주는 즐거움 때문에 번지점프와 같은 가상의 위기 상황을 만들어 일부러 몰입을 추구하기도 한다."
            "그런데 이왕이면 업무나 학습 활동에 몰입하여 높은 기량도 쌓고 즐거움도 얻는 게 더 좋을 것이다."
            "이것은 삶에서 대단히 중요한 문제이고 이 방법을 터득하면 삶의 행복을 찾을 수 있다."
            )

In [2]:
pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 45.3 MB/s eta 0:00:00


In [3]:
import konlpy
from konlpy.tag import Okt, Mecab

In [4]:
tokenizer=Okt()
print(tokenizer.morphs(sentence))

['사람', '은', '누구', '나', '몰입', '할', '수', '있는', '능력', '을', '가지', '고', '있다', '.', '위기', '상황', '에서', '몰입', '하기도', '하고', ',', '몰입', '이', '주는', '즐거움', '때문', '에', '번지점프', '와', '같은', '가상', '의', '위기', '상황', '을', '만들어', '일부러', '몰입', '을', '추구', '하기도', '한다', '.', '그런데', '이왕이면', '업무', '나', '학습', '활동', '에', '몰입', '하여', '높은', '기량', '도', '쌓고', '즐거움', '도', '얻는', '게', '더', '좋을', '것', '이다', '.', '이', '것', '은', '삶', '에서', '대단히', '중요한', '문제', '이고', '이', '방법', '을', '터득', '하면', '삶', '의', '행복', '을', '찾을', '수', '있다', '.']


In [5]:
world_set = list(set(sentence)) # 중복 문자 제거

## 문제(1): 각 문자에 정수 인코딩 (공백도 하나의 원소로 포함)
# 코드 시행할 때마다 인덱스 붙는게 계속 달라짐.
# world_set을 sorted() 함수를 써서 정렬하면 인덱스가 정해진 순서로 고정됨

#vocab = {##}
vocab = {world: idx for idx, world in enumerate(world_set)} # 딕셔너리 자료형에 key = 단어, value: 인덱스로 저장

In [6]:
vocab_size = len(vocab)
print('문자 집합 크기 : {}'.format(vocab_size))

문자 집합 크기 : 85


In [7]:
hidden_size = vocab_size
sequence_length = 15
learning_rate = 0.01

In [8]:
#5. seqence 길이 단위 자르기

# 데이터 구성을 위한 리스트
x_data = []
y_data = []

## 문제(2): 반복문 내에서의 인덱싱을 사용하여 sequence_length 값 단위로 샘플을 잘라 데이터 만들기, y_str은 x_str은 한 칸씩 쉬프트된 sequnce

for i in range(0, len(sentence) - sequence_length):
  x_str = sentence[i:i+sequence_length]
  y_str = sentence[i+1: i+sequence_length + 1 ]
  print(i, x_str, "->", y_str)

  # x_str과 y_str이 문자집합에 해당하는 인덱스를 각각 x_data, y_data에 append
  x_data.append([vocab[c] for c in x_str])
  y_data.append([vocab[d] for d in y_str])

0 사람은 누구나 몰입할 수 있 -> 람은 누구나 몰입할 수 있는
1 람은 누구나 몰입할 수 있는 -> 은 누구나 몰입할 수 있는 
2 은 누구나 몰입할 수 있는  ->  누구나 몰입할 수 있는 능
3  누구나 몰입할 수 있는 능 -> 누구나 몰입할 수 있는 능력
4 누구나 몰입할 수 있는 능력 -> 구나 몰입할 수 있는 능력을
5 구나 몰입할 수 있는 능력을 -> 나 몰입할 수 있는 능력을 
6 나 몰입할 수 있는 능력을  ->  몰입할 수 있는 능력을 가
7  몰입할 수 있는 능력을 가 -> 몰입할 수 있는 능력을 가지
8 몰입할 수 있는 능력을 가지 -> 입할 수 있는 능력을 가지고
9 입할 수 있는 능력을 가지고 -> 할 수 있는 능력을 가지고 
10 할 수 있는 능력을 가지고  ->  수 있는 능력을 가지고 있
11  수 있는 능력을 가지고 있 -> 수 있는 능력을 가지고 있다
12 수 있는 능력을 가지고 있다 ->  있는 능력을 가지고 있다.
13  있는 능력을 가지고 있다. -> 있는 능력을 가지고 있다.위
14 있는 능력을 가지고 있다.위 -> 는 능력을 가지고 있다.위기
15 는 능력을 가지고 있다.위기 ->  능력을 가지고 있다.위기 
16  능력을 가지고 있다.위기  -> 능력을 가지고 있다.위기 상
17 능력을 가지고 있다.위기 상 -> 력을 가지고 있다.위기 상황
18 력을 가지고 있다.위기 상황 -> 을 가지고 있다.위기 상황에
19 을 가지고 있다.위기 상황에 ->  가지고 있다.위기 상황에서
20  가지고 있다.위기 상황에서 -> 가지고 있다.위기 상황에서 
21 가지고 있다.위기 상황에서  -> 지고 있다.위기 상황에서 몰
22 지고 있다.위기 상황에서 몰 -> 고 있다.위기 상황에서 몰입
23 고 있다.위기 상황에서 몰입 ->  있다.위기 상황에서 몰입하
24  있다.위기 상황에서 몰입하 -> 있다.위기 상황에서 몰입하기
25 있다.위기 상황에서 몰입하기 -> 다.위기 상황에서 몰입하기도
26 다.위기 상황에서 몰입하기도 -> 

In [11]:
x_data_np = np.array(x_data)
x_one_hot = np.eye(len(vocab))[x_data_np] #len(vocab) 크기의 identity matrix 형성. 인덱스별로 대응 -> one hot 인코딩!

##7. 입력 데이터, 레이블데이터 텐서로 변환

## 문제(5) : x_one_hot과 y_data 텐서로 변환 : 둘 다 같은 형식의 텐서로 변환하면 될까?? (FloatTensor, LongTesor 중 맞는 것은?)
X = torch.FloatTensor(x_one_hot)## 신경망의 가중치와 편향이 일반적으로 부동소수점 형태로 저장되어 있음.
Y = torch.LongTensor(y_data) # 레이블은 주로 클래스 또는 카테고리를 나타냄, 어느정도 관례!

In [12]:
class Net(torch.nn.Module):
  def __init__(self, input_dim, hidden_dim, layers):
    super(Net, self).__init__()
    self.rnn = torch.nn.RNN(input_dim,
                            hidden_dim,
                            layers,
                            batch_first = True,
                            )
    self.fc = torch.nn.Linear(hidden_dim, input_dim) # ??? fully conneceted layer 차원 어떻게 구성하는지, 아직 원리 이해 x

  def forward(self, x):
    # First RNN layer
    x, _ = self.rnn(x)

    # Second RNN layer
    x, _ = self.rnn(x)

    # Fully connected layer
    x = self.fc(x)

    return x

In [13]:
net = Net(vocab_size, hidden_size, 2)

In [14]:
##12. loss function
criterion = torch.nn.CrossEntropyLoss()

##13. optimizer
optimizer = optim.Adam(net.parameters(), learning_rate)

In [15]:
##15. Training 시작

for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    ##문제(7) : outputs, Y 형태 그대로 넣으면 안되죠. view 함수를 이용해 loss값을 계산해봅시다.
    #loss = criterion(outputs.view(##, ##), Y.view(##))
    loss = criterion(outputs.view(-1, vocab_size), Y.view(-1)) # criterion 손실함수 - 크로스엔트로피
    loss.backward()
    optimizer.step()

    #16. 예측결과 확인
    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0: # 처음에는 예측 결과를 전부 가져오기
            predict_str += ''.join([world_set[t] for t in result])
        else: # 그 다음에는 마지막 글자만 반복 추가
            predict_str += world_set[result[-1]]

    print(predict_str)

런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런런누런런런런런런런런런런런런런런
                                                                                                                                                                                                          
                                                                                                                                                                                                          
                                                                                                                                                                                                          
                                                                                                                                                                                            

In [16]:
predict_str

'람은 누구나 몰입할 수 있는 능력을 가지고 있다.위기 상황에서 몰입하기도 하고, 몰입이 주는 즐거움 때문에 번지점프와 같은 가상의 위기 상황을 만들어 일부러 몰입을 추구하기도 한다.그런데 이왕이면 업무나 학습 활동에 몰입하여 높은 기량도 쌓고 즐거움도 얻는 게 더 좋을 것이다.이것은 삶에서 대단히 중요한 문제이고 이 방법을 터득하면 삶의 행복을 찾을 수 있다.'